In [1]:
import shutup
import warnings
import time
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification
from transformers import logging
import pandas as pd
import json

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
    )


def prediction_bert_category(arr,companyname,instanceid):
    start = time.time()
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings('ignore')
    shutup.please()
    logging.set_verbosity_error()
    arr = arr['question']
    category_file_name = "category" + companyname + str(instanceid) + ".txt"
    with open(category_file_name) as f:
        name = f.readlines()
    categories = json.loads(name[0])
    categories = {int(k):v for k,v in categories.items()}
    model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(categories),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )
    Bert_model_Name = "BertModel" + companyname + str(instanceid) 
    model = torch.load(Bert_model_Name)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    finalcategory = []
    for i in range(len(arr)):
        review_text = arr[i]
        encoded_review = tokenizer.encode_plus(
        review_text,
        max_length=256,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
        )
        input_ids = encoded_review['input_ids'].to(device)
        attention_mask = encoded_review['attention_mask'].to(device)
        output = model(input_ids, attention_mask)
        final_pred = torch.tensor(output.logits)
        sentimentIndex = int(torch.argmax(final_pred))
        finalcategoryiment = categories[sentimentIndex]
        finalcategory.append(finalcategoryiment)
    return finalcategory





In [3]:
data = pd.read_csv("sendwasim.csv")
data = data[['question','category']]
data
dct = {}
for i in range(len(data)):
    if data.iloc[i]['category'] in dct.keys():
        dct[data.iloc[i]['category']] = dct[data.iloc[i]['category']] + 1
    else:
        dct[data.iloc[i]['category']] = 0
for key in dct:
    if dct[key] < 2:
        index_names = data[data['category'] == key].index
        data.drop(index_names, inplace = True)
data.index = range(len(data))
dct = {}
dct['question'] = data['question'].values
dct['category'] = data['category'].values
datanew = pd.DataFrame(dct)



In [4]:
datanew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  263 non-null    object
 1   category  268 non-null    object
dtypes: object(2)
memory usage: 4.3+ KB


In [5]:
datanew = datanew.dropna()


In [6]:
datanew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 267
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  263 non-null    object
 1   category  263 non-null    object
dtypes: object(2)
memory usage: 6.2+ KB


In [8]:
datanew = ne

,question,category
0,gm,Greeting
1,what is data science?,course
2,i want talk with agent,Greetings
3,bye,Greetings
4,how do i reset my password?,FAQ
...,...,...
263,hi,greeting
264,want to know about emi payment options,payment
265,contact sales,course
266,english books,#


In [7]:
companyname = "Amazon"
instanceid = 2
output = prediction_bert_category(datanew,companyname,instanceid)

KeyError: 32